In [3]:
"""
NOTE: ON RE-RUNNING THIS SCRIPT, THE OLD OHLC CSVS WILL BE OVERWRITTEN
PLEASE STORE THE OLD CSVS SOMEWHERE SAFE THEN YOU CAN PROCEED
"""

from fyers_apiv3.FyersWebsocket import data_ws
from fyers_apiv3 import fyersModel
from datetime import datetime,timedelta
import pandas as pd
import threading
from pytz import timezone
import time
import pandas as pd
from csv import DictWriter
import os

#generate trading session
client_id = open("../client_ID.txt",'r').read()
access_token = open("../access_token.txt",'r').read()

timeframe = 1   # timeframe in minutes

vol_data = {}

csv_data = {} 

# Initialize the FyersModel instance with your client_id, access_token, and enable async mode
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")

timeframe_counter = 1

prev_vol = 0

def onmessage(message):

    global timeframe_counter
    global timeframe
    global prev_vol

    print("Response", message)
    ms = message['exch_feed_time']
    curr_time = datetime.fromtimestamp(ms)
    print("message exch_feed_time:", curr_time)

    if curr_time.second == 0:
        timeframe_counter += 1
    
    if timeframe_counter == timeframe:
        for symbol in vol_data:
            try:
                # find ohlc and save in csv
                curr_vol = message['vol_traded_today']

                #print(vol_data[symbol])
                csv_dict = {'minute': str(curr_time), 'symbol': str(symbol),
                            'total_volume': int(curr_vol), f'{timeframe}_min_vol': int(curr_vol - prev_vol)}
                
                print(curr_vol - prev_vol)
                prev_vol = curr_vol


                if csv_data.get(symbol) == None:
                    csv_data[symbol] = []

                csv_data[symbol].append(csv_dict)
                df_dictionary = pd.DataFrame(csv_data[symbol])
                df_dictionary.to_csv(f'{symbol.replace(":", "_")}.csv')

                #generate_csv(symbol.replace(":", "_"), ['symbol', 'high', 'low', 'open', 'close', 'minute'], csv_dict)

                print("CSV MADE")
                print(csv_dict)

            except Exception as e:
                print(e)
            
            vol_data[symbol] = []

        timeframe_counter = 0

    else:
        if vol_data.get(message['symbol']) == None:
            vol_data[message['symbol']] = []
        
        vol_data[message['symbol']].append(float(message['vol_traded_today']))
        print(vol_data)


def onerror(message):
    print("Error:", message)

def onclose(message):
    print("Connection closed:", message)

def onopen():
    data_type = "SymbolUpdate"

    # Subscribe to the specified symbols and data type
    symbols = ['MCX:CRUDEOIL25SEPFUT']
    fyers.subscribe(symbols=symbols, data_type=data_type)

    # Keep the socket running to receive real-time data
    fyers.keep_running()

def generate_csv(csv_name, field_names, dict):

    if not os.path.isfile(f'{csv_name}.csv'):
        with open(f'{csv_name}.csv', 'a', newline='') as obj:
            dictwriter_object = DictWriter(obj, fieldnames=field_names)
            dictwriter_object.writeheader()
            dictwriter_object.writerow(dict)
            obj.close()
    else:
        with open(f'{csv_name}.csv', 'a', newline='') as obj:
            dictwriter_object = DictWriter(obj, fieldnames=field_names)
            dictwriter_object.writerow(dict)
            obj.close()

# Create a FyersDataSocket instance with the provided parameters
fyers = data_ws.FyersDataSocket(
    access_token=access_token,       # Access token in the format "appid:accesstoken"
    log_path="",                     # Path to save logs. Leave empty to auto-create logs in the current directory.
    litemode=False,                  # Lite mode disabled. Set to True if you want a lite response.
    write_to_file=False,              # Save response in a log file instead of printing it.
    reconnect=True,                  # Enable auto-reconnection to WebSocket on disconnection.
    on_connect=onopen,               # Callback function to subscribe to data upon connection.
    on_close=onclose,                # Callback function to handle WebSocket connection close events.
    on_error=onerror,                # Callback function to handle WebSocket errors.
    on_message=onmessage             # Callback function to handle incoming messages from the WebSocket.
)

# Establish a connection to the Fyers WebSocket

def main():
    print("Inside main()")
    fyers.connect()

main()

Inside main()
Response {'type': 'cn', 'code': 200, 'message': 'Authentication done', 's': 'ok'}
Error: KeyError: The key 'exch_feed_time' is missing in the response.
Response {'type': 'ful', 'code': 200, 'message': 'Full Mode On', 's': 'ok'}
Error: KeyError: The key 'exch_feed_time' is missing in the response.


Response {'type': 'sub', 'code': 200, 'message': 'Subscribed', 's': 'ok'}
Error: KeyError: The key 'exch_feed_time' is missing in the response.
Response {'ltp': 5689.0, 'vol_traded_today': 8209, 'last_traded_time': 1756473993, 'exch_feed_time': 1756474021, 'bid_size': 24, 'ask_size': 24, 'bid_price': 5686.0, 'ask_price': 5689.0, 'last_traded_qty': 5, 'tot_buy_qty': 647, 'tot_sell_qty': 868, 'avg_trade_price': 5668.74, 'low_price': 5627.0, 'high_price': 5698.0, 'lower_ckt': 0, 'upper_ckt': 0, 'open_price': 5628.0, 'prev_close_price': 5619.0, 'type': 'sf', 'symbol': 'MCX:CRUDEOIL25SEPFUT', 'ch': 70.0, 'chp': 1.2458}
message exch_feed_time: 2025-08-29 18:57:01
Response {'ltp': 5689.0, 'vol_traded_today': 8210, 'last_traded_time': 1756474022, 'exch_feed_time': 1756474022, 'bid_size': 3, 'ask_size': 23, 'bid_price': 5687.0, 'ask_price': 5689.0, 'last_traded_qty': 1, 'tot_buy_qty': 663, 'tot_sell_qty': 874, 'avg_trade_price': 5668.74, 'low_price': 5627.0, 'high_price': 5698.0, 'lower_ckt': 0

In [1]:
from fyers_apiv3.FyersWebsocket import data_ws

client_id = open("../client_ID.txt",'r').read()
access_token = open("../access_token.txt",'r').read()

def onmessage(message):
    """
    Callback function to handle incoming messages from the FyersDataSocket Websocket
    Parameters:
            message (dict): The received message from the WebSocket 
    """
    print("Response:", message)
    print("Symbol:",message['symbol'],  "Volume: ",message['vol_traded_today'])


def onerror(message):
    """
    Callback function to handle Websocket errors
    Parameters:
            message (dict): The error message received from the WebSocket 
    """

    print("Error", message)


def onclose(message):
    """
    Callback function to handle Websocket connection close events. 
    """

    print("Connection Closed", message)


def onopen():
    """
    Callback function to subscribe to data type and symbols upon Websocket connection. 
    """
    # Specify the data type and symbols you want to subscribe to
    data_type = "SymbolUpdate"

    # Subscribe to the specified symbols and data types
    symbols = ['MCX:CRUDEOIL25SEPFUT']
    fyers.subscribe(symbols=symbols, data_type=data_type)

    # Keep the socket running to receive real-time data
    fyers.keep_running()


# Create a FyersDataSocket instance with the provided parameters
fyers = data_ws.FyersDataSocket(
    access_token=access_token,      # Access token in format "appid:accesstoken"
    log_path="",                    # Path to save logs. Leave empty to auto-create logs in the current directory
    litemode=False,                 # Lite mode disabled. Set to True if you want a lite response.
    write_to_file=False,            # Save response in a log file instead of printing it.
    reconnect=True,                 # Enable auto-reconnection to WebSocket on disconnection.
    on_connect=onopen,              # Callback function to subscribe to data upon connection.
    on_close=onclose,               # Callback function to handle WebSocket connection close events.
    on_error=onerror,               # Callback function to handle WebSocket errors.
    on_message=onmessage            # Callback function to handle incoming messages from the WebSocket.
)

fyers.connect()

Response: {'type': 'cn', 'code': 200, 'message': 'Authentication done', 's': 'ok'}
Error KeyError: The key 'symbol' is missing in the response.
Response: {'type': 'ful', 'code': 200, 'message': 'Full Mode On', 's': 'ok'}
Error KeyError: The key 'symbol' is missing in the response.


Response: {'type': 'sub', 'code': 200, 'message': 'Subscribed', 's': 'ok'}
Error KeyError: The key 'symbol' is missing in the response.
Response: {'ltp': 5570.0, 'vol_traded_today': 7227, 'last_traded_time': 1756390520, 'exch_feed_time': 1756390520, 'bid_size': 15, 'ask_size': 7, 'bid_price': 5569.0, 'ask_price': 5570.0, 'last_traded_qty': 1, 'tot_buy_qty': 528, 'tot_sell_qty': 966, 'avg_trade_price': 5600.0, 'low_price': 5569.0, 'high_price': 5631.0, 'lower_ckt': 0, 'upper_ckt': 0, 'open_price': 5602.0, 'prev_close_price': 5618.0, 'type': 'sf', 'symbol': 'MCX:CRUDEOIL25SEPFUT', 'ch': -48.0, 'chp': -0.8544}
Symbol: MCX:CRUDEOIL25SEPFUT Volume:  7227
Response: {'ltp': 5572.0, 'vol_traded_today': 7236, 'last_traded_time': 1756390521, 'exch_feed_time': 1756390522, 'bid_size': 36, 'ask_size': 23, 'bid_price': 5570.0, 'ask_price': 5573.0, 'last_traded_qty': 1, 'tot_buy_qty': 562, 'tot_sell_qty': 956, 'avg_trade_price': 5599.96, 'low_price': 5569.0, 'high_price': 5631.0, 'lower_ckt': 0, 'upp

In [6]:
from fyers_apiv3.FyersWebsocket import data_ws
from datetime import datetime
import pandas as pd
import pytz
import os
from collections import deque

# Load credentials
client_id = open("../client_ID.txt", 'r').read().strip()
access_token = open("../access_token.txt", 'r').read().strip()

# Global storage
prev_volumes = {}    # last cumulative volume per symbol
minute_data = {}     # current tracking of minute volume
csv_data = {}        # all rows saved to CSV
last_15 = {}         # store last 15 minutes volumes per symbol (deque)

# Timezone
ist = pytz.timezone("Asia/Kolkata")

MY_WHATSAPP = "+91XXXXXXXXXX"

def send_alert(symbol, minute, one_min_vol, avg_15):
    msg = (f"🚨 Volume Breakout Alert 🚨\n\n"
           f"Symbol: {symbol}\n"
           f"Minute: {minute}\n"
           f"1-min Volume: {one_min_vol}\n"
           f"15-min Avg: {round(avg_15,2)}\n"
           f"Condition: {one_min_vol} ≥ 10 × {round(avg_15,2)}")

    try:
        # send instant WhatsApp message (opens web.whatsapp in background)
        kit.sendwhatmsg_instantly(
            phone_no=MY_WHATSAPP,
            message=msg,
            wait_time=15,  # seconds to wait for WhatsApp web to load
            tab_close=True
        )
        print(f"[ALERT SENT] {symbol} {minute}")
    except Exception as e:
        print("Failed to send WhatsApp:", e)


def onmessage(message):
    global prev_volumes, minute_data, last_15, csv_data

    try:
        symbol = message["symbol"]
        exch_time = datetime.fromtimestamp(message["exch_feed_time"], tz=ist)
        curr_minute = exch_time.replace(second=0, microsecond=0)

        # Current cumulative volume
        cum_vol = int(message["vol_traded_today"])

        # First tick for this symbol
        if symbol not in prev_volumes:
            prev_volumes[symbol] = cum_vol
            minute_data[symbol] = {"minute": curr_minute, "volume": 0}
            last_15[symbol] = deque(maxlen=15)
            return

        # Reset case (session rollover)
        if cum_vol < prev_volumes[symbol]:
            prev_volumes[symbol] = cum_vol
            minute_data[symbol] = {"minute": curr_minute, "volume": 0}
            last_15[symbol].clear()
            return

        # Delta volume
        delta_vol = cum_vol - prev_volumes[symbol]
        prev_volumes[symbol] = cum_vol

        # Same minute -> accumulate
        if curr_minute == minute_data[symbol]["minute"]:
            minute_data[symbol]["volume"] += delta_vol
        else:
            # Finalize previous minute
            finalized = {
                "minute": minute_data[symbol]["minute"],
                "symbol": symbol,
                "1_min_volume": minute_data[symbol]["volume"]
            }

            # Save into CSV
            if symbol not in csv_data:
                csv_data[symbol] = []
            csv_data[symbol].append(finalized)
            df = pd.DataFrame(csv_data[symbol])
            df.to_csv(f'{symbol.replace(":", "_")}.csv', index=False)

            print("Saved:", finalized)

            # Push into 15-min buffer
            last_15[symbol].append(finalized["1_min_volume"])

            # --- ALERT LOGIC ---
            if len(last_15[symbol]) >= 15:
                avg_15 = sum(last_15[symbol]) / len(last_15[symbol])
                if finalized["1_min_volume"] >= 15 * avg_15 and avg_15 > 0:
                    print(f"🚨 ALERT: {symbol} 1-min vol {finalized['1_min_volume']} "
                          f"is 15× avg {avg_15:.2f} at {finalized['minute']}")
                    # <-- WhatsApp alert function can be called here -->
                        send_alert(symbol, curr_minute, one_min_vol, avg_15)

            # Start new minute
            minute_data[symbol] = {"minute": curr_minute, "volume": delta_vol}

    except Exception as e:
        print("Error in onmessage:", e)


def onerror(message):
    print("Error:", message)


def onclose(message):
    print("Connection closed:", message)


def onopen():
    data_type = "SymbolUpdate"
    symbols = ['MCX:CRUDEOIL25SEPFUT', 'MCX:SILVER25SEPFUT']  # add more here
    fyers.subscribe(symbols=symbols, data_type=data_type)
    fyers.keep_running()


# Create websocket
fyers = data_ws.FyersDataSocket(
    access_token=access_token,
    log_path="",
    litemode=False,
    write_to_file=False,
    reconnect=True,
    on_connect=onopen,
    on_close=onclose,
    on_error=onerror,
    on_message=onmessage
)


def main():
    print("Starting WebSocket...")
    fyers.connect()


if __name__ == "__main__":
    main()


Starting WebSocket...
Error in onmessage: 'symbol'
Error in onmessage: 'symbol'


Error in onmessage: 'symbol'
Saved: {'minute': datetime.datetime(2025, 8, 29, 19, 23, tzinfo=<DstTzInfo 'Asia/Kolkata' IST+5:30:00 STD>), 'symbol': 'MCX:SILVER25SEPFUT', '1_min_volume': 3}
Saved: {'minute': datetime.datetime(2025, 8, 29, 19, 23, tzinfo=<DstTzInfo 'Asia/Kolkata' IST+5:30:00 STD>), 'symbol': 'MCX:CRUDEOIL25SEPFUT', '1_min_volume': 1}
Saved: {'minute': datetime.datetime(2025, 8, 29, 19, 24, tzinfo=<DstTzInfo 'Asia/Kolkata' IST+5:30:00 STD>), 'symbol': 'MCX:CRUDEOIL25SEPFUT', '1_min_volume': 20}
Saved: {'minute': datetime.datetime(2025, 8, 29, 19, 24, tzinfo=<DstTzInfo 'Asia/Kolkata' IST+5:30:00 STD>), 'symbol': 'MCX:SILVER25SEPFUT', '1_min_volume': 19}
Saved: {'minute': datetime.datetime(2025, 8, 29, 19, 25, tzinfo=<DstTzInfo 'Asia/Kolkata' IST+5:30:00 STD>), 'symbol': 'MCX:CRUDEOIL25SEPFUT', '1_min_volume': 5}
Saved: {'minute': datetime.datetime(2025, 8, 29, 19, 25, tzinfo=<DstTzInfo 'Asia/Kolkata' IST+5:30:00 STD>), 'symbol': 'MCX:SILVER25SEPFUT', '1_min_volume': 5}
Sav